### PyEmma Featurizer Support

In [1]:
import openpathsampling as ps
import openpathsampling.storage as st

In [2]:
storage = st.Storage('trajectory.nc', mode='a')

Import a PyEmma Coordinates Module

In [4]:
import pyemma.coordinates as coor

Using of pyemma featurizers or general other complex code requires a little trick to be storable. Since storing of code only works if we are not dependend on the context (scope) we need to wrap the construction of our featurizer in a function, that gets all it needs from the global scope as a parameter

In [5]:
def pyemma_generator(f):
    f.add_inverse_distances(f.pairs(f.select_Backbone()))

In [6]:
cv = ps.collectivevariable.CV_PyEMMA_Featurizer('pyemma', pyemma_generator, topology=storage.template.topology)

Now use this featurizer generating function to build a collective variable out of it. All we need for that is a name as usual, the generating function, the list of parameters - here only the topology and at best a test snapshot, a template.

In [7]:
cv(storage.trajectories[0]);

Let's save it to the storage

In [8]:
cv.save(storage)

and apply the featurizer to a trajectory

In [9]:
cv(storage.trajectories[2]);

Sync to make sure the cache is written to the netCDF file.

In [10]:
storage.sync_all()

In [11]:
cv(storage.snapshots.all());

In [12]:
storage.cvs.sync()

In [13]:
storage.sync_all()

In [14]:
py_cv = storage.cvs['pyemma']

In [15]:
nc_var = storage.variables['snapshots_cv_%d_values' % storage.idx(py_cv)]

In [16]:
assert(nc_var.shape[1] == 15)
print nc_var.shape[1]

15


In [17]:
assert(nc_var.var_type == 'numpy.float32')
print nc_var.var_type

numpy.float32


In [18]:
#! ignore
print storage.variables['cvs_json'][:]

[ u'{"_cls": "CV_MDTraj_Function", "_dict": {"store_cache": true, "name": "psi", "f": {"_module": "mdtraj.geometry.dihedral", "_name": "compute_dihedrals"}, "scalarize_numpy_singletons": true, "wrap_numpy_array": true, "kwargs": {"indices": [[6, 8, 14, 16]]}, "requires_lists": true, "time_reversible": true}}'
 u'{"_cls": "CV_MDTraj_Function", "_dict": {"store_cache": true, "name": "phi", "f": {"_module": "mdtraj.geometry.dihedral", "_name": "compute_dihedrals"}, "scalarize_numpy_singletons": true, "wrap_numpy_array": true, "kwargs": {"indices": [[4, 6, 8, 14]]}, "requires_lists": true, "time_reversible": true}}'
 u'{"_cls": "CV_Volume", "_dict": {"volume": {"_obj": "volumes", "_idx": 0}, "store_cache": true, "name": "StateA"}}'
 u'{"_cls": "CV_Volume", "_dict": {"volume": {"_obj": "volumes", "_idx": 25}, "store_cache": true, "name": "StateB"}}'
 u'{"_cls": "CV_Volume", "_dict": {"volume": {"_obj": "volumes", "_idx": 26}, "store_cache": true, "name": "StateX"}}'
 u'{"_cls": "CV_PyEMMA_F

In [19]:
py_emma_feat = storage.vars['cvs_json'][5]

In [20]:
erg = py_emma_feat(storage.snapshots);

In [21]:
print erg[0::10,1]

[ 4.05474138  4.08869648  4.02007341  4.04071331  4.04777765  4.12398911
  3.97356868  3.96233702  3.99706006  4.0870719   4.03449965  4.01160955
  3.95180297  4.03023672  3.93415093  4.0085144   3.97549605  3.94362783
  3.97176313  3.99334955  3.99146152  4.06641769  4.16512966  4.04492378
  4.15058517  4.04605961  3.94790721  3.95720506  4.02582121  4.08820391
  3.98605466  3.90670562  3.81981516  4.01113605  3.88823342  4.10035563
  4.14985943  3.91612959  3.98040414  3.89736295  4.12943125  3.93787169
  3.91101813  3.95772362]
